In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/Data_Klaim.csv
/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/sample_submission.csv
/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/Data_Polis.csv


# DATA FOUNDATION

In [2]:
# ============================================================
# STAGE 1 v5 — CLEAN MINIMAL FOUNDATION
# Payment/Service switch • No lag drop • No exposure drift
# ============================================================

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

BASE_PATH = "/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/"

klaim = pd.read_csv(BASE_PATH + "Data_Klaim.csv")
polis = pd.read_csv(BASE_PATH + "Data_Polis.csv")

# =============================
# CLEAN COLUMNS
# =============================

def clean_columns(df):
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_", regex=False)
        .str.replace("/", "_", regex=False)
        .str.replace("-", "_", regex=False)
    )
    return df

klaim = clean_columns(klaim)
polis = clean_columns(polis)

klaim = klaim.drop_duplicates().reset_index(drop=True)
polis = polis.drop_duplicates().reset_index(drop=True)

# =============================
# DATE PARSING
# =============================

for col in klaim.columns:
    if "tanggal" in col:
        klaim[col] = pd.to_datetime(klaim[col], errors="coerce")

for col in polis.columns:
    if "tanggal" in col:
        polis[col] = pd.to_datetime(polis[col], errors="coerce")

# =============================
# BASIC CLEANING
# =============================

klaim = klaim.dropna(subset=["nomor_polis"])
klaim["nominal_klaim_yang_disetujui"] = klaim["nominal_klaim_yang_disetujui"].fillna(0)

df = klaim.merge(polis, on="nomor_polis", how="left")

# =============================
# MONTH AXIS (VERY IMPORTANT)
# =============================

MONTH_TYPE = "PAYMENT"   # <<< GANTI KE PAYMENT

if MONTH_TYPE == "SERVICE":
    month_col = "tanggal_pasien_masuk_rs"
elif MONTH_TYPE == "PAYMENT":
    month_col = "tanggal_pembayaran_klaim"
else:
    raise ValueError("MONTH_TYPE must be SERVICE or PAYMENT")

df = df.dropna(subset=[month_col]).copy()
df["year_month"] = df[month_col].dt.to_period("M")

# =============================
# MONTHLY AGGREGATION
# =============================

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .reset_index()
      .sort_values("year_month")
      .reset_index(drop=True)
)

monthly["severity"] = (
    monthly["total_claim"] /
    monthly["frequency"].replace(0,np.nan)
)

print("Month type:", MONTH_TYPE)
print("Monthly shape:", monthly.shape)
print("Unique months:", monthly["year_month"].nunique())
print("\nSTAGE 1 v5 READY")


Month type: PAYMENT
Monthly shape: (24, 4)
Unique months: 24

STAGE 1 v5 READY


In [3]:
# ============================================================
# STAGE 1 v4 — USE PAYMENT DATE (CRITICAL TEST)
# ============================================================

df["year_month"] = df["tanggal_pembayaran_klaim"].dt.to_period("M")

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .reset_index()
      .sort_values("year_month")
      .reset_index(drop=True)
)

print(monthly)

   year_month  frequency   total_claim
0     2024-01          8  1.283162e+08
1     2024-02         92  2.684171e+09
2     2024-03         97  3.809944e+09
3     2024-04        221  9.281203e+09
4     2024-05        233  1.103847e+10
5     2024-06        221  1.127720e+10
6     2024-07        205  1.159773e+10
7     2024-08        285  1.895989e+10
8     2024-09        250  1.484250e+10
9     2024-10        242  1.114198e+10
10    2024-11        365  1.740396e+10
11    2024-12        295  1.409901e+10
12    2025-01        293  1.697253e+10
13    2025-02        183  9.559585e+09
14    2025-03        234  1.494105e+10
15    2025-04        184  7.538943e+09
16    2025-05        201  9.628068e+09
17    2025-06        204  1.617766e+10
18    2025-07        272  1.862361e+10
19    2025-08        245  1.546896e+10
20    2025-09        197  1.041073e+10
21    2025-10         58  4.900102e+09
22    2025-11          3  1.356322e+08
23    2025-12          2  1.366003e+08


# TIME-SERIES DATASET ENGINEERING

In [4]:
# ============================================================
# STAGE 2 v3 — STABLE SEGMENT SHARE PANEL
# Plan-Level Only • Share-Based • Short Series Safe
# ============================================================

import numpy as np
import pandas as pd

# ============================================================
# ENSURE PLAN CODE EXISTS
# ============================================================

if "plan_code" not in df.columns:
    df["plan_code"] = "UNKNOWN"

df["plan_code"] = df["plan_code"].fillna("UNKNOWN")

# ============================================================
# BUILD MONTHLY PLAN-LEVEL PANEL
# ============================================================

seg_monthly = (
    df.groupby(["year_month", "plan_code"])
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .reset_index()
      .sort_values(["plan_code","year_month"])
      .reset_index(drop=True)
)

# ============================================================
# MERGE GLOBAL TOTAL (UNTUK SHARE)
# ============================================================

global_monthly = (
    df.groupby("year_month")
      .agg(total_global=("nominal_klaim_yang_disetujui","sum"))
      .reset_index()
)

seg_monthly = seg_monthly.merge(global_monthly, on="year_month", how="left")

# ============================================================
# SHARE OF TOTAL (STABLE TARGET)
# ============================================================

seg_monthly["share_total"] = (
    seg_monthly["total_claim"] /
    seg_monthly["total_global"].replace(0,np.nan)
)

# ============================================================
# SMOOTH SHARE (SHORT SERIES SAFE)
# ============================================================

seg_monthly["share_roll3"] = (
    seg_monthly.groupby("plan_code")["share_total"]
    .transform(lambda x: x.rolling(3, min_periods=1).mean())
)

# ============================================================
# LOG DOMAIN (OPTIONAL)
# ============================================================

seg_monthly["log_total"] = np.log1p(seg_monthly["total_claim"])
seg_monthly["log_freq"]  = np.log1p(seg_monthly["frequency"])

# ============================================================
# LAG FEATURES (SAFE)
# ============================================================

for col in ["log_total","log_freq","share_roll3"]:
    seg_monthly[f"{col}_lag1"] = \
        seg_monthly.groupby("plan_code")[col].shift(1)
    
    seg_monthly[f"{col}_lag2"] = \
        seg_monthly.groupby("plan_code")[col].shift(2)

# ============================================================
# DROP EARLY NA (MINIMAL LOSS)
# ============================================================

seg_model = seg_monthly.dropna().reset_index(drop=True)

# ============================================================
# FINAL CHECK
# ============================================================

print("SEGMENT PANEL SHAPE:", seg_model.shape)
print("Unique plan codes:", seg_model["plan_code"].nunique())
print("Months per plan (min):", 
      seg_model.groupby("plan_code")["year_month"].nunique().min())

print("\nSTAGE 2 v3 — STABLE SEGMENT SHARE PANEL READY")


SEGMENT PANEL SHAPE: (62, 15)
Unique plan codes: 3
Months per plan (min): 19

STAGE 2 v3 — STABLE SEGMENT SHARE PANEL READY


# MODEL DEVELOPMENT

In [5]:
# ============================================================
# STAGE 3 FINAL — SERVICE OPTIMIZED VERSION
# Stable modeling for leaderboard
# ============================================================

import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

def mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# ============================================================
# BUILD SERVICE MONTHLY
# ============================================================

df["year_month"] = df["tanggal_pasien_masuk_rs"].dt.to_period("M")

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .reset_index()
      .sort_values("year_month")
      .reset_index(drop=True)
)

monthly["severity"] = monthly["total_claim"] / monthly["frequency"]

monthly["log_freq"]  = np.log1p(monthly["frequency"])
monthly["log_sev"]   = np.log1p(monthly["severity"])
monthly["log_total"] = np.log1p(monthly["total_claim"])

print("Total SERVICE months:", len(monthly))

# ============================================================
# SPLIT (LAST 5 MONTHS)
# ============================================================

train = monthly.iloc[:-5].copy()
valid = monthly.iloc[-5:].copy()

H = len(valid)

# ============================================================
# MODELING (DAMPED + LOG DOMAIN)
# ============================================================

model_freq = ExponentialSmoothing(
    train["log_freq"],
    trend="add",
    damped_trend=True,
    seasonal=None
).fit()

model_sev = ExponentialSmoothing(
    train["log_sev"],
    trend="add",
    damped_trend=True,
    seasonal=None
).fit()

model_total = ExponentialSmoothing(
    train["log_total"],
    trend="add",
    damped_trend=True,
    seasonal=None
).fit()

# ============================================================
# FORECAST
# ============================================================

freq_fc = np.expm1(model_freq.forecast(H))
sev_fc  = np.expm1(model_sev.forecast(H))

total_direct = np.expm1(model_total.forecast(H))
total_mul = freq_fc * sev_fc

# Ensemble (lebih stabil di SERVICE)
total_fc = 0.5 * total_direct + 0.5 * total_mul

# ============================================================
# RESULTS
# ============================================================

print("\n==============================")
print("MAPE Frequency :", round(mape(valid["frequency"], freq_fc),4))
print("MAPE Severity  :", round(mape(valid["severity"], sev_fc),4))
print("MAPE Total     :", round(mape(valid["total_claim"], total_fc),4))
print("Estimated Score:", round(np.mean([
    mape(valid["frequency"], freq_fc),
    mape(valid["severity"], sev_fc),
    mape(valid["total_claim"], total_fc)
]),4))
print("==============================")


Total SERVICE months: 19

MAPE Frequency : 5.9765
MAPE Severity  : 4.5584
MAPE Total     : 6.9296
Estimated Score: 5.8215


# TOTAL CLAIM OPTIMIZATION & VALIDATION, OPTUNA

In [6]:
# ============================================================
# STAGE 4 PRO REVISED — FULL MULTI PARAM OPTIMIZATION
# Optimize freq, sev, total separately + ensemble weight
# ============================================================

!pip install -q optuna

import optuna
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

# ============================================================
# METRIC
# ============================================================

def mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask]))

def kaggle_score(freq_t, freq_p, sev_t, sev_p, total_t, total_p):
    return (
        mape(freq_t, freq_p) +
        mape(sev_t, sev_p) +
        mape(total_t, total_p)
    ) / 3

# ============================================================
# BUILD SERVICE MONTHLY
# ============================================================

df["year_month"] = df["tanggal_pasien_masuk_rs"].dt.to_period("M")

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .reset_index()
      .sort_values("year_month")
      .reset_index(drop=True)
)

monthly["severity"] = monthly["total_claim"] / monthly["frequency"]

monthly["log_freq"]  = np.log1p(monthly["frequency"])
monthly["log_sev"]   = np.log1p(monthly["severity"])
monthly["log_total"] = np.log1p(monthly["total_claim"])

train_full = monthly.iloc[:-4].copy()
valid_full = monthly.iloc[-4:].copy()

H = 4

# ============================================================
# OBJECTIVE
# ============================================================

def objective(trial):

    # freq params
    a_f = trial.suggest_float("alpha_freq", 0.1, 0.9)
    b_f = trial.suggest_float("beta_freq", 0.01, 0.4)
    p_f = trial.suggest_float("phi_freq", 0.7, 0.99)

    # sev params
    a_s = trial.suggest_float("alpha_sev", 0.1, 0.9)
    b_s = trial.suggest_float("beta_sev", 0.01, 0.4)
    p_s = trial.suggest_float("phi_sev", 0.7, 0.99)

    # total params
    a_t = trial.suggest_float("alpha_total", 0.1, 0.9)
    b_t = trial.suggest_float("beta_total", 0.01, 0.4)
    p_t = trial.suggest_float("phi_total", 0.7, 0.99)

    weight = trial.suggest_float("ensemble_weight", 0.0, 1.0)

    sim_df = train_full.copy()

    freq_preds = []
    sev_preds = []
    total_preds = []

    for step in range(H):

        try:
            model_freq = ExponentialSmoothing(
                sim_df["log_freq"],
                trend="add",
                damped_trend=True
            ).fit(
                smoothing_level=a_f,
                smoothing_trend=b_f,
                damping_trend=p_f,
                optimized=False
            )

            model_sev = ExponentialSmoothing(
                sim_df["log_sev"],
                trend="add",
                damped_trend=True
            ).fit(
                smoothing_level=a_s,
                smoothing_trend=b_s,
                damping_trend=p_s,
                optimized=False
            )

            model_total = ExponentialSmoothing(
                sim_df["log_total"],
                trend="add",
                damped_trend=True
            ).fit(
                smoothing_level=a_t,
                smoothing_trend=b_t,
                damping_trend=p_t,
                optimized=False
            )

            freq_fc = np.expm1(model_freq.forecast(1).iloc[0])
            sev_fc  = np.expm1(model_sev.forecast(1).iloc[0])
            total_direct = np.expm1(model_total.forecast(1).iloc[0])

        except:
            freq_fc = sim_df["frequency"].iloc[-1]
            sev_fc  = sim_df["severity"].iloc[-1]
            total_direct = sim_df["total_claim"].iloc[-1]

        total_mul = freq_fc * sev_fc
        total_final = weight * total_direct + (1-weight) * total_mul

        freq_preds.append(freq_fc)
        sev_preds.append(sev_fc)
        total_preds.append(total_final)

        new_row = {
            "year_month": None,
            "frequency": freq_fc,
            "severity": sev_fc,
            "total_claim": total_final,
            "log_freq": np.log1p(freq_fc),
            "log_sev": np.log1p(sev_fc),
            "log_total": np.log1p(total_final)
        }

        sim_df = pd.concat([sim_df, pd.DataFrame([new_row])], ignore_index=True)

    return kaggle_score(
        valid_full["frequency"], freq_preds,
        valid_full["severity"], sev_preds,
        valid_full["total_claim"], total_preds
    )

# ============================================================
# RUN OPTUNA
# ============================================================

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=200)

print("\nBest Params:")
print(study.best_params)
print("Best 4M Kaggle Score:", round(study.best_value*100,4), "%")

[I 2026-02-19 05:50:17,284] A new study created in memory with name: no-name-576f24b1-68f5-4b88-b124-1e2070506e66
[I 2026-02-19 05:50:17,311] Trial 0 finished with value: 0.11807072032774657 and parameters: {'alpha_freq': 0.42864505548029097, 'beta_freq': 0.15546019357551183, 'phi_freq': 0.7620903789176418, 'alpha_sev': 0.6481727708009328, 'beta_sev': 0.18488022238429452, 'phi_sev': 0.771364838857228, 'alpha_total': 0.21009095356738863, 'beta_total': 0.0558968213474799, 'phi_total': 0.9057222404523629, 'ensemble_weight': 0.8910948923111021}. Best is trial 0 with value: 0.11807072032774657.
[I 2026-02-19 05:50:17,338] Trial 1 finished with value: 0.07112526288168063 and parameters: {'alpha_freq': 0.6100501385328532, 'beta_freq': 0.2254710458987247, 'phi_freq': 0.9608008968551063, 'alpha_sev': 0.24446248254301164, 'beta_sev': 0.030383312796681182, 'phi_sev': 0.9655514920621882, 'alpha_total': 0.7859986625863992, 'beta_total': 0.35337712622520656, 'phi_total': 0.7784247050268677, 'ensembl


Best Params:
{'alpha_freq': 0.5931675323787786, 'beta_freq': 0.26077232742527734, 'phi_freq': 0.7003496001336191, 'alpha_sev': 0.11675015831407387, 'beta_sev': 0.36906318518430437, 'phi_sev': 0.9414307908276309, 'alpha_total': 0.7503460002242532, 'beta_total': 0.1402871731781296, 'phi_total': 0.7218674803434785, 'ensemble_weight': 0.00426250589993758}
Best 4M Kaggle Score: 4.8958 %


# TEST PREDICTION & KAGGLE SUBMISSION

In [7]:
# ============================================================
# STAGE 5 FINAL FIXED — NO NaN VERSION
# ============================================================

import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

BASE_PATH = "/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/"
sample_sub = pd.read_csv(BASE_PATH + "sample_submission.csv")

# ============================================================
# DOMAIN
# ============================================================

USE_PAYMENT = True  # ganti kalau perlu

if USE_PAYMENT:
    df["year_month"] = df["tanggal_pembayaran_klaim"].dt.to_period("M")
else:
    df["year_month"] = df["tanggal_pasien_masuk_rs"].dt.to_period("M")

# ============================================================
# BEST PARAMS (PASTE ASLI DARI STAGE 4)
# ============================================================

BEST_PARAMS = {
    "alpha_freq": 0.3,
    "beta_freq": 0.05,
    "phi_freq": 0.95,
    "alpha_sev": 0.4,
    "beta_sev": 0.08,
    "phi_sev": 0.97,
    "alpha_total": 0.25,
    "beta_total": 0.07,
    "phi_total": 0.94,
    "ensemble_weight": 0.55
}

# ============================================================
# BUILD MONTHLY
# ============================================================

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .reset_index()
      .sort_values("year_month")
      .reset_index(drop=True)
)

monthly["severity"] = monthly["total_claim"] / monthly["frequency"].replace(0, np.nan)

monthly["log_freq"]  = np.log1p(monthly["frequency"])
monthly["log_sev"]   = np.log1p(monthly["severity"])
monthly["log_total"] = np.log1p(monthly["total_claim"])

# ============================================================
# PREPARE FUTURE MONTHS
# ============================================================

sample_sub["year"]  = sample_sub["id"].str.split("_").str[0]
sample_sub["month"] = sample_sub["id"].str.split("_").str[1]
sample_sub["month_key"] = sample_sub["year"] + "-" + sample_sub["month"]

future_periods = (
    pd.PeriodIndex(sample_sub["month_key"], freq="M")
      .unique()
      .sort_values()
)

future_df = monthly.copy()
predictions = {}

# ============================================================
# RECURSIVE FORECAST FOR ALL REQUESTED MONTHS
# ============================================================

for period in future_periods:

    model_freq = ExponentialSmoothing(
        future_df["log_freq"],
        trend="add",
        damped_trend=True
    ).fit(
        smoothing_level=BEST_PARAMS["alpha_freq"],
        smoothing_trend=BEST_PARAMS["beta_freq"],
        damping_trend=BEST_PARAMS["phi_freq"],
        optimized=False
    )

    model_sev = ExponentialSmoothing(
        future_df["log_sev"],
        trend="add",
        damped_trend=True
    ).fit(
        smoothing_level=BEST_PARAMS["alpha_sev"],
        smoothing_trend=BEST_PARAMS["beta_sev"],
        damping_trend=BEST_PARAMS["phi_sev"],
        optimized=False
    )

    model_total = ExponentialSmoothing(
        future_df["log_total"],
        trend="add",
        damped_trend=True
    ).fit(
        smoothing_level=BEST_PARAMS["alpha_total"],
        smoothing_trend=BEST_PARAMS["beta_total"],
        damping_trend=BEST_PARAMS["phi_total"],
        optimized=False
    )

    freq_fc = max(np.expm1(model_freq.forecast(1).iloc[0]), 0)
    sev_fc  = max(np.expm1(model_sev.forecast(1).iloc[0]), 0)
    total_direct = max(np.expm1(model_total.forecast(1).iloc[0]), 0)

    total_mul = freq_fc * sev_fc

    w = BEST_PARAMS["ensemble_weight"]
    total_final = max(w * total_direct + (1 - w) * total_mul, 0)

    key_total = f"{period.year}_{str(period.month).zfill(2)}_Total_Claim"
    key_freq  = f"{period.year}_{str(period.month).zfill(2)}_Claim_Frequency"
    key_sev   = f"{period.year}_{str(period.month).zfill(2)}_Claim_Severity"

    predictions[key_total] = total_final
    predictions[key_freq]  = freq_fc
    predictions[key_sev]   = sev_fc

    new_row = {
        "year_month": period,
        "frequency": freq_fc,
        "severity": sev_fc,
        "total_claim": total_final,
        "log_freq": np.log1p(freq_fc),
        "log_sev": np.log1p(sev_fc),
        "log_total": np.log1p(total_final)
    }

    future_df = pd.concat([future_df, pd.DataFrame([new_row])], ignore_index=True)

# ============================================================
# BUILD SUBMISSION
# ============================================================

submission = sample_sub.copy()
submission["value"] = submission["id"].map(predictions)

# fallback safety (tidak boleh ada NaN)
submission["value"] = submission["value"].fillna(0)

submission = submission[["id","value"]]
submission.to_csv("submission.csv", index=False)

print("Submission created — NO NaN VERSION")
print(submission.head())


Submission created — NO NaN VERSION
                        id         value
0  2025_08_Claim_Frequency  1.997265e+01
1   2025_08_Claim_Severity  6.529127e+07
2      2025_08_Total_Claim  1.404461e+09
3  2025_09_Claim_Frequency  1.848141e+01
4   2025_09_Claim_Severity  6.648550e+07


In [8]:
print(submission.head(9)) ## 74 persen

                        id         value
0  2025_08_Claim_Frequency  1.997265e+01
1   2025_08_Claim_Severity  6.529127e+07
2      2025_08_Total_Claim  1.404461e+09
3  2025_09_Claim_Frequency  1.848141e+01
4   2025_09_Claim_Severity  6.648550e+07
5      2025_09_Total_Claim  1.277275e+09
6  2025_10_Claim_Frequency  1.716306e+01
7   2025_10_Claim_Severity  6.766476e+07
8      2025_10_Total_Claim  1.172338e+09


In [9]:
import pandas as pd
import numpy as np

# =====================================
# BUILD BOTH AXIS
# =====================================

df["service_month"] = df["tanggal_pasien_masuk_rs"].dt.to_period("M")
df["payment_month"] = df["tanggal_pembayaran_klaim"].dt.to_period("M")

service_monthly = (
    df.groupby("service_month")
      .agg(freq=("claim_id","count"),
           total=("nominal_klaim_yang_disetujui","sum"))
      .reset_index()
      .sort_values("service_month")
)

payment_monthly = (
    df.groupby("payment_month")
      .agg(freq=("claim_id","count"),
           total=("nominal_klaim_yang_disetujui","sum"))
      .reset_index()
      .sort_values("payment_month")
)

print("===== BASIC INFO =====")
print("Service months:", service_monthly["service_month"].min(), "→", service_monthly["service_month"].max())
print("Payment months:", payment_monthly["payment_month"].min(), "→", payment_monthly["payment_month"].max())
print()

print("Total months (Service):", len(service_monthly))
print("Total months (Payment):", len(payment_monthly))
print()

# =====================================
# TAIL CHECK
# =====================================

print("===== LAST 6 MONTHS (SERVICE) =====")
print(service_monthly.tail(6))
print()

print("===== LAST 6 MONTHS (PAYMENT) =====")
print(payment_monthly.tail(6))
print()

# =====================================
# VOLATILITY CHECK
# =====================================

print("===== VOLATILITY =====")
print("Service freq std/mean:", 
      service_monthly["freq"].std() / service_monthly["freq"].mean())

print("Payment freq std/mean:", 
      payment_monthly["freq"].std() / payment_monthly["freq"].mean())
print()

# =====================================
# SEASONAL CHECK (AVG PER MONTH)
# =====================================

payment_monthly["month"] = payment_monthly["payment_month"].dt.month

seasonal = payment_monthly.groupby("month")["total"].mean()

print("===== AVG TOTAL PER CALENDAR MONTH =====")
print(seasonal.sort_values())


===== BASIC INFO =====
Service months: 2024-01 → 2025-07
Payment months: 2024-01 → 2025-12

Total months (Service): 19
Total months (Payment): 24

===== LAST 6 MONTHS (SERVICE) =====
   service_month  freq         total
13       2025-02   246  1.748054e+10
14       2025-03   230  1.367924e+10
15       2025-04   207  1.090143e+10
16       2025-05   234  1.190261e+10
17       2025-06   227  1.231230e+10
18       2025-07   260  1.322321e+10

===== LAST 6 MONTHS (PAYMENT) =====
   payment_month  freq         total
18       2025-07   272  1.862361e+10
19       2025-08   245  1.546896e+10
20       2025-09   197  1.041073e+10
21       2025-10    58  4.900102e+09
22       2025-11     3  1.356322e+08
23       2025-12     2  1.366003e+08

===== VOLATILITY =====
Service freq std/mean: 0.10975628930790748
Payment freq std/mean: 0.5146225591902012

===== AVG TOTAL PER CALENDAR MONTH =====
month
2     6.121878e+09
12    7.117807e+09
10    8.021042e+09
4     8.410073e+09
1     8.550421e+09
11    8.76